In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=True
# yu.slName='20240218_ETMC'

In [ ]:
ensembles=['cA211.530.24','cA2.09.48']

ens2data={}
for ens in ensembles:
    path={
        'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_c/data/cA211.530.24_NST_a_daint.h5',
        'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_c/data/cA2.09.48_NST_c_booster.h5',
        }[ens]
    ens2data[ens]=yu.load(path)
    print('Ncfg='+str(yu.deepKey(ens2data[ens]['2pt'],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data['2pt'],2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

In [ ]:
rec=lat.ZS; Ncfg=100
t=np.random.normal(loc=rec[0],scale=rec[1]*np.sqrt(Ncfg),size=Ncfg)
print(rec)
yu.jackknife(t)


In [ ]:
# updateEns(ensembles[0])
# t=data['VEV']['j']['id_j+'][0]; print(t)
# t=data['VEV']['j']['g5_j-'][0]; print(t)
# t=data['VEV']['pi0f'][0]; print(t)

# updateEns(ensembles[1])
# t=data['VEV']['j']['id_j+'][0]; print(t)
# t=data['VEV']['j']['g5_j-'][0]; print(t)
# t=data['VEV']['pi0f'][0]; print(t)

In [ ]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'remove_pi0VEV':True, 
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    res=np.zeros([Ncfg,Tpack],dtype=complex)
    if opa+'_'+opb in data['2pt']:
        t=data['2pt'][opa+'_'+opb]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt']:
        t=data['2pt'][opb+'_'+opa]; 
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0))
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    res=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)
    if opa+'_'+opb in data['3pt']:
        t=data['3pt'][opa+'_'+opb][insert]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}
        if opa+'_'+opb not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opa+'_'+opb][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

def remove_pi0VEV(Ctij,pi0VEV,opas,opbs):
    if not flags['remove_pi0VEV']:
        return Ctij
    
    factor=yu.coeff['g;0,0,1;G1;N1pi0,a;l1;N,pi'][0][1]
    
    ka=None # index for N(n)pi0(0)
    for i,op in enumerate(opas):
        if yu.getNpar(op)==1:
            qa=i # index for N(n)
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            ka=i
            factor_a = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
    kb=None
    for i,op in enumerate(opbs):
        if yu.getNpar(op)==1:
            qb=i
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            kb=i
            factor_b = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
            
    res=Ctij.copy()
    if ka is not None:
        res[:,ka,:]-=factor_a*Ctij[:,qa,:]
    if kb is not None:
        res[:,:,kb]-=np.conj(factor_b)*Ctij[:,:,qb]
    if ka is not None and kb is not None:
        res[:,ka,kb]+=factor_a*np.conj(factor_b)*Ctij[:,qa,qb]
    
    # print(factor_a,factor_b)
    # print(pi0VEV)
        
    return res

In [ ]:
# choices

ens2choice={
    ('cA211.530.24','0,0,0'):
        {
            'choice':[(t-4,t) for t in range(12,12+1)],
            },
    ('cA2.09.48','0,0,0'):
        {
            'choice':[(t-5,t) for t in range(13,13+1)],
            },
    ('cA211.530.24','0,0,1'):
        {
            'choice':[(t-4,t) for t in range(13,13+1)],
            },
    ('cA2.09.48','0,0,1'):
        {
            'choice':[(t-5,t) for t in range(13,13+1)],
            },
}


dts=range(1,11)
def run_t0_dep2(pt,ofs,diags=yu.diags_all,ylim1=None,ylim2=None,ylim3=None,ylim4=None,ylim5=None,label=None):
    Nop=len(ofs)
    
    fig, axs = yu.getFigAxs(3*2,len(dts),sharex=True,sharey='row')
    yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors','cA24: W','cA48: Energy','cA48: E-vectors','cA48: W'])
    yu.addColHeader(axs,['dt='+str(dt) for dt in dts])
    
    eV_func=np.abs
    W_func=np.abs
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops,diags),data['VEV']['pi0f']]
        
        for i_dt,dt in enumerate(dts):
            irow=i_ens*3+1; icol=i_dt
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dt,tList=None if dt>=4 else range(Tpack-3))
                eVecs=yu.renormalize_eVecs(eVecs)
                t=eVecs[:,0,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func,sl_key=('t0_dep2'+label,ens,dt,'eVec'))
            for ind in range(Nop):
                tmin=dt+1; tmax=len(mean[ind])
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
                if 1!=None:
                    for t in range(tmin,tmax):
                        if (t-dt,t) in ens2choice[(ens,pt)]['choice']:
                            axs[irow,icol].errorbar(t*lat.a,mean[ind][t],err[ind][t],color='b',fmt=['s','d','o'][ind])
            axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
            
            irow=i_ens*3+2
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dt,tList=None if dt>=4 else range(Tpack-3))
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func,sl_key=('t0_dep2'+label,ens,dt,'W'))
            for ind in range(0,1):
                tmin=dt+1; tmax=len(mean[ind])
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
                if 1!=None:
                    for t in range(tmin,tmax):
                        if (t-dt,t) in ens2choice[(ens,pt)]['choice']:
                            axs[irow,icol].errorbar(t*lat.a,mean[ind][t],err[ind][t],color='b',fmt=['s','d','o'][ind])
            axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])
        
        # t0List=[t0 for t0,t in ens2choice[(ens,pt)]['choice']]
        # tList=[t for t0,t in ens2choice[(ens,pt)]['choice']]
        # for ind in range(1,Nop):
        #     def func(dat):
        #         t=yu.meanDeep(dat)
        #         t2pt,tVEVpi0f=t
        #         t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        #         (eVals,eVecs)=yu.GEVP(t2pt,t0List,tList=tList)
        #         eVecs=yu.renormalize_eVecs(eVecs)
        #         return eVecs[:,0,ind]
        
        
        # def func(dat):
        #     t=yu.meanDeep(dat)
        #     t2pt,tVEVpi0f=t
        #     t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        #     (eVals,eVecs)=yu.GEVP(t2pt,t0List,tList=tList)
        #     eVecs=yu.renormalize_eVecs(eVecs)
        #     eVecs_inv_tin=np.linalg.inv(eVecs)
        #     W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
        #     return W
            

            
    plt.tight_layout()
    plt.savefig('fig/t0_dep2'+label+'.pdf')
    plt.close()
    print(label)

# run_t0_dep2('0,0,0',[('a','p'),('N1pi1,a','12'),],label='_0_N1pi1')
# run_t0_dep2('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),],label='_1_N1pi0_N0pi1')
run_t0_dep2('0,0,0',[('a','p'),('N1pi1,a','12'),],ylim2=[0,0.5],ylim3=[0,0.05],ylim4=[0,0.4],ylim5=[0,0.2],label='_0_N1pi1')
run_t0_dep2('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),],ylim2=[0,0.1],ylim3=[0,0.025],ylim4=[0,0.07],ylim5=[0,0.04],label='_1_N1pi0_N0pi1')

In [ ]:
# 2pt E-energy fits

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))

ens2info={
    ('cA211.530.24','0,0,0'):
        {
            'tmins_1st':range(2,13+1),
            'tmins_2st':range(1,9+1),
            'tmins_3st':range(1,2+1),
            },
    ('cA2.09.48','0,0,0'):
        {
            'tmins_1st':range(6,18+1),
            'tmins_2st':range(1,9+1),
            'tmins_3st':range(1,2+1),
            },
    ('cA211.530.24','0,0,1'):
        {
            'tmins_1st':range(2,13+1),
            'tmins_2st':range(1,9+1),
            'tmins_3st':range(1,2+1),
            },
    ('cA2.09.48','0,0,1'):
        {
            'tmins_1st':range(6,18+1),
            'tmins_2st':range(1,9+1),
            'tmins_3st':range(1,2+1),
            },
}

ens2tfRange={}
ens2p0={}

def run2pt(pt,ofs,ens):
    updateEns(ens)
    xUnit=lat.a; yUnit=lat.aInv
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
    fig.suptitle((ens,pt))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    
    t0List=[t0 for t0,t in ens2choice[(ens,pt)]['choice']]
    tList=[t for t0,t in ens2choice[(ens,pt)]['choice']]
    
    for case in ['noGEVP','GEVP']:
        mfc={'noGEVP':'white','GEVP':None}[case]
        ops=yu.getops(pt,'l1',ofs)
        if case=='noGEVP':
            ops=[ops[0]]
        dat=[get2ptMat(ops),data['VEV']['pi0f']] 
        
        # get tfmax
        def func_2pt(dat):
            t2pt,tVEVpi0f=yu.meanDeep(dat)
            t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
            _,eVecs=yu.GEVP(t2pt,t0List,tList=tList)
            eVecs=yu.renormalize_eVecs(eVecs)
            eVec=np.mean(eVecs[:,0],axis=0)
            t=np.einsum('i,tij,j->t',np.conj(eVec),t2pt,eVec)
            return np.real(t)
        mean,err,cov=yu.jackknife(dat,func_2pt,sl_key=('run2pt',ens,pt,case,1))
        t=np.abs(err[0]/mean[0])
        tfmax=Tpack
        for tf in range(1,Tpack):
            if t[tf]>0.2:
                tfmax=tf
                print(tfmax)
                break
        
        def func(dat):
            t=func_2pt(dat)
            t=np.log(t/np.roll(t,-1))
            return t
        mean,err,cov=yu.jackknife(dat,func,sl_key=('run2pt',ens,pt,case,2))
        tmin=2; tmax=Tpack-1
        ax1.errorbar(np.arange(tmin,tmax)*xUnit,mean[0][tmin:tmax]*yUnit,yerr=err[0][tmin:tmax]*yUnit,color='orange',mfc=mfc)
        
        # initial pars
        pars0_initial=[1000/yUnit, 8e-8, 1000/yUnit, 2, 2000/yUnit, 1]
        
        # 1st fit
        pars0=pars0_initial[:2]
        fits=[]
        temp_ax2=[]
        for tmin_fit in ens2info[(ens,pt)]['tmins_1st']:
            tfs=np.arange(tmin_fit,tfmax)
            def func(dat):
                t=func_2pt(dat)[tfs]
                return t
            def fitfunc(E0,c0):
                return func_C2pt_1st(tfs,E0,c0)
            est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof,pars0=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('run2pt',ens,pt,case,3,tmin_fit))
            # print(tmin_fit,pars_mean)
            ax2.errorbar([tmin_fit*xUnit],est_mean[0]*yUnit,yerr=est_err[0]*yUnit,color='r',mfc=mfc)
            temp_ax2.append([chi2R_mean,tmin_fit,est_mean[0],est_err[0],'r'])
            # ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[0]-est_err[0]-0.005)*yUnit),color='r')
            fits.append((est_mean,est_err,chi2R_mean,Ndof))
            
            # axs[irow,icol+2].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,pars_mean[8]-pars_err[8]-shift_chi2),color=color)
        
        pars0=pars0_initial[:4]
        fits=[]
        temp_ax3=[]
        def estimator(pars):
            return pars+[0,0,pars[0],0]
        for tmin_fit in ens2info[(ens,pt)]['tmins_2st']:
            # print('2st',tmin_fit)
            tfs=np.arange(tmin_fit,tmax+1)
            def func(dat):
                t=func_2pt(dat)[tfs]
                return t
            def fitfunc(E0,c0,dE1,rc1):
                return func_C2pt_2st(tfs,E0,c0,dE1,rc1)
            est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof,pars0=yu.fit(dat,func,fitfunc,estimator=estimator,pars0=pars0,sl_key=('run2pt',ens,pt,case,4,tmin_fit))
            # print(ens,pt,case,tmin_fit,pars0,est_mean)
            # print(tmin_fit,pars_mean[0],pars_err[0])
            ax2.errorbar([tmin_fit*xUnit],est_mean[0]*yUnit,yerr=est_err[0]*yUnit,color='g',mfc=mfc)
            temp_ax2.append([chi2R_mean,tmin_fit,est_mean[0],est_err[0],'g'])
            # ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[0]-est_err[0]-0.005)*yUnit),color='g')
            ax3.errorbar([tmin_fit*xUnit],est_mean[2]*yUnit,yerr=est_err[2]*yUnit,color='g',mfc=mfc)
            temp_ax3.append([chi2R_mean,tmin_fit,est_mean[2],est_err[2],'g'])
            # ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[2]-est_err[2]-0.005)*yUnit),color='g')
            fits.append((est_mean,est_err,chi2R_mean,Ndof))
        est_mean_MA,est_err_MA,probs=yu.modelAvg(fits)
        tmean,terr=est_mean_MA[0],est_err_MA[0]; 
        ylim1=np.array([tmean-terr*10,tmean+terr*40])*yUnit
        ylim2=np.array([tmean-terr*10,tmean+terr*20])*yUnit
        tmean,terr=est_mean_MA[2],est_err_MA[2]; 
        ylim3=np.array([tmean-terr*5,tmean+terr*5])*yUnit
        ax1.set_ylim(ylim1); ax2.set_ylim(ylim2); ax3.set_ylim(ylim3)
        ens2tfRange[(ens,pt,case)]=[ens2info[(ens,pt)]['tmins_2st'][probs.argmax()],tfmax]
        t=fits[4][0]; t[2]=t[2]-t[0]
        ens2p0[(ens,pt,case)]=t
        
        ysgn={'noGEVP':1/2,'GEVP':-1}[case]
        yshift=(ylim2[1]-ylim2[0])/10
        for chi2R_mean,tmin_fit,est_mean,est_err,color in temp_ax2:
            ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean+ysgn*(est_err+yshift/yUnit))*yUnit),color=color)
        yshift=(ylim3[1]-ylim3[0])/10
        for chi2R_mean,tmin_fit,est_mean,est_err,color in temp_ax3:
            ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean+ysgn*(est_err+yshift/yUnit))*yUnit),color=color)
        
        # 3st fit
        # pars0=[0.44, 8e-8, 0.33, 1.75, 0.7, 1]
        # # pars0[:4]=estimatoInv(pars_mean_MA)
        # fits=[]
        # def estimator(pars):
        #     return pars+[0,0,pars[0],0,pars[0],0]
        # for tmin_fit in ens2info[(ens,pt)]['tmins_3st']:
        #     # print('3st',tmin_fit)
        #     tfs=np.arange(tmin_fit,tmax+1)
        #     def func(dat):
        #         t=func_2pt(dat)[tfs]
        #         return t
        #     def fitfunc(E0,c0,dE1,rc1,dE2,rc2):
        #         return func_C2pt_3st(tfs,E0,c0,dE1,rc1,dE2,rc2)
        #     est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,pars0=pars0)
        #     ax2.errorbar([tmin_fit*xUnit],est_mean[0]*yUnit,yerr=est_err[0]*yUnit,color='b',mfc=mfc)
        #     ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[0]-est_err[0]-0.005)*yUnit),color='b')
        #     ax3.errorbar([tmin_fit*xUnit],est_mean[2]*yUnit,yerr=est_err[2]*yUnit,color='b',mfc=mfc)
        #     ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[2]-est_err[2]-0.005)*yUnit),color='b')
        #     fits.append((est_mean,est_err,chi2R_mean,Ndof))
        
    plt.tight_layout()
    plt.savefig('fig/run2pt_'+ens+'_'+pt+'_.pdf')
    plt.close()
    print(ens,pt)

for ens in ensembles:
    run2pt('0,0,0',[('a','p'),('N1pi1,a','12'),],ens)
    run2pt('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),],ens)
ens2tfRange

In [ ]:
# utitlies

func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C3pt_2st=lambda tf,tc,E0a,E0b,a00,dE1a,dE1b,ra01,ra10,ra11: a00*np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(1 + ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc))

flags['dt2pt']=False
flags['dt3pt']=True

ylimDic={
    'GS+_00':[6,22],'GA_00_zz':[0.5,1.6],
    'J5_11':[-2.5,2.5],'GA_11_tz':[0.5,1.6],'GA_11_zz':[0.5,1.6],'GA_11_xx':[0.5,1.6],
    'G5b_01':[0,25],'GA_01':[0.8,1.3],'GPb_01_tz':[10,60],'GPb_01_zz':[10,60],
    'rPCAC_1_Ptz':[0.2,1.5],'rPCAC_1_Pzz':[0.2,1.5],
}

FF2simone={
    'GA_00_zz':'GA_0',
    'GA_11_tz':'GA_0',
    'GA_11_zz':'GA_0',
    'GA_11_xx':'GA_0',
    'GA_01':'GA_1',
    'GPb_01_tz':'GPb_1',
    'GPb_01_zz':'GPb_1',
    'G5b_01':'G5b_1',
    'rPCAC_1_Ptz':'rPCAC_1',
    'rPCAC_1_Pzz':'rPCAC_1',
}

simone={
    'GA_0':[1.245,0.031],
    'GA_1':[1.129,0.027],
    'GPb_1':[48.9,2.8],
    'G5b_1':[12.4,0.5],
    'rPCAC_1':[1.06,0.05],
}

tfs=[10,12,14]
def get_dt(eVecsa_ltni,eVecsb_ltni):
    eVecsa_ltni=yu.renormalize_eVecs(eVecsa_ltni)
    # eVecsa_ltni=eVecsa_ltni/np.diagonal(eVecsa_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsa_lni=np.mean(eVecsa_ltni,axis=1)
    eVecsa_inv_lin=np.linalg.inv(eVecsa_lni)
    Wa_l=1/(eVecsa_lni[:,0,0]*eVecsa_inv_lin[:,0,0])-1
    eVeca_li=eVecsa_lni[:,0]
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    eVecsb_ltni=yu.renormalize_eVecs(eVecsb_ltni)
    # eVecsb_ltni=eVecsb_ltni/np.diagonal(eVecsb_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsb_lni=np.mean(eVecsb_ltni,axis=1)
    eVecsb_inv_lin=np.linalg.inv(eVecsb_lni)
    Wb_l=1/(eVecsb_lni[:,0,0]*eVecsb_inv_lin[:,0,0])-1
    eVecb_li=eVecsb_lni[:,0]
    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)


In [ ]:
fitres_PCAC={}
def runFit_PCAC():
    ptofas=('0,0,0',[('a','p'),('N1pi1,a','12')])
    ptofbs=('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),])
    
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        xUnit=lat.a
        
        t0Lista=[t0 for t0,t in ens2choice[(ens,pta)]['choice']]
        tLista=[t for t0,t in ens2choice[(ens,pta)]['choice']]
        t0Listb=[t0 for t0,t in ens2choice[(ens,ptb)]['choice']]
        tListb=[t for t0,t in ens2choice[(ens,ptb)]['choice']]
        
        mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
        t_Fpi=92.9/lat.aInv
        t_aQ2=(qk**2-(EN-mN)**2)
        t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
        t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
        t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
        
        t_A2pNN_0=mN/t_Fpi
        t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
        t_52pNN_1=(lat.amu/lat.ZP[0])*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
        t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
        
        t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA[0]
        t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS[0]; t_G5b_01=t_G5_01 * t_52pNN_1
        t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA[0]; t_GP_01_tz_A=-(2*mN)/(EN-mN)
        t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA[0]; t_GP_01_zz_A=-(2*mN)/(EN-mN)

        # t_PCAC_G5=(lat.amu/lat.ZP[0])/mN
        # t_PCAC_GP=t_aQ2/(4*mN**2)
        
        pipss=[
            [(t_G5b_01,'g5_j-','Pz')],
            [(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
            [(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
            [(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
        ]
        
        # pips={
        #     'GS+_00':[(lat.ZP[0],'id_j+','P0')],
        #     'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
        #     'J5_11':[(lat.ZS[0],'g5_j-','Pz')],
        #     'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
        #     'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
        #     'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
            
        #     'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
        #     'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
        #     'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
        #     'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
            
        #     # 'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
        #     # 'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA[0],'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA[0],'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP[0]*lat.ZS[0],'g5_j-','Pz')],
            
        #     # 'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        #     # 'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        
        #     # 'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        #     # 'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
            
        #     # 'rPCAC2_1':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),(2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
        # }[FF]
         
        # ren={
        #     'GS+_00':lat.ZP,'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
        #     'G5b_01':lat.ZS,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,
        # }[FF]
        def errRen(mean,err,ren):
            tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
            tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
            return tmean,np.sqrt(np.diag(tcov))
                
        proj2mat={
            'P0':np.array([[1,0],[0,1]]),
            'Px':np.array([[0,1],[1,0]]),
            'Py':np.array([[0,-1j],[1j,0]]),
            'Pz':np.array([[1,0],[0,-1]]),
        }
        
        for i_diags,diags,str_diag in [(0,yu.diags_jLoopless,'loopless'),(1,yu.diags_all,'all')]:
            irow=i_ens*2+i_diags
            needsVEV=False
            
            for case in ['noGEVP','GEVP']:
                pta,ofas=ptofas; ptb,ofbs=ptofbs
                if case=='noGEVP':
                    ofas=[ofas[0]]; ofbs=[ofbs[0]]
                
                def t_get(tf,t_insert):
                    t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                        for lb in ['l1','l2']] for la in ['l1','l2']]
                    return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                
                dat3pt=[[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs] for pips in pipss]
                dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                        get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                datVEVj=[data['VEV']['j']['id_j+' if 0 in ['GS+_00'] else 'g5_j-']]
                datVEVpi0f=[data['VEV']['pi0f']]
                dat=[dat3pt,[dat2pt,datVEVj,datVEVpi0f]] 
                
                AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
                
                def func_base(dat):
                    t=yu.meanDeep(dat)
                    t3pt,temp=t
                    t2pt,tVEVj,tVEVpi0f=temp
                    
                    t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                    t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                    t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                    t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                    
                    for ind in range(len(t3pt)):
                        for ind2 in range(len(t3pt[ind])):
                            for ia,la in enumerate(['l1','l2']):
                                for ib,lb in enumerate(['l1','l2']): 
                                    t3pt[ind][ind2][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][ind2][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                    
                    _,eVecs_al1=yu.GEVP(t2pt[0],t0Lista,tList=tLista)
                    _,eVecs_al2=yu.GEVP(t2pt[1],t0Lista,tList=tLista)
                    _,eVecs_bl1=yu.GEVP(t2pt[2],t0Listb,tList=tListb)
                    _,eVecs_bl2=yu.GEVP(t2pt[3],t0Listb,tList=tListb)

                    eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                    eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni)
                    
                    t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                    t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                    
                    t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                    t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                    
                    # if needsVEV:
                    #     proj=proj2mat[pips[0][2]]
                    #     t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                    #         np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                            
                    def t_get2(i,i_pips,pips):
                        res=0
                        for i_pip,pip in enumerate(pips):
                            prefactor,insert,proj=pip
                            proj=proj2mat[proj]
                            proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                            res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pips][i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                        return res
                    t=[t2pta_t,t2ptb_t]+[ t_get2(i,i_pips,pips) for i_pips,pips in enumerate(pipss) for i,tf in enumerate(tfs)]
                
                    t=yu.prefactorDeep(t,1)
                    return t
                
                pars0=np.concatenate([ens2p0[(ens,pta,case)],ens2p0[(ens,ptb,case)],\
                    [np.mean(ylimDic['G5b_01']),1,1],[np.mean(ylimDic['GA_01']),1,1],[np.mean(ylimDic['GPb_01_tz']),1,1],[1,1]])
                fits=[]
                for datmin in range(1,6):
                    print(ens,str_diag,case,datmin,end='                              \r')
                    datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                    tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                    tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                    tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                    tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                    def func(dat):
                        t=func_base(dat)
                        t[0]=t[0][tfs_2pta_fit]
                        t[1]=t[1][tfs_2ptb_fit]
                        for i_pips in range(len(pipss)):
                            for i,tf in enumerate(tfs):
                                t[i_pips*len(tfs)+i+2]=t[i_pips*len(tfs)+i+2][tcs_3pt_fit[tf]-1]
                        return t
                    def fitfunc(E0a,c0a,dE1a,rc1a,E0b, c0b,dE1b,rc1b,g_5b,ra01_5b, ra10_5b,g_A,ra01_A,ra10_A,g_Pb, ra01_Pb_tz,ra10_Pb_tz,ra01_Pb_zz,ra10_Pb_zz):
                        t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                        t_2ptb=func_C2pt_2st(tfs_2ptb_fit,E0b,c0b,dE1b,rc1b)
                        t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0b,g*np.sqrt(c0a*c0b),dE1a,dE1b,ra01,ra10,0) for g,ra01,ra10 in \
                            [(g_5b,ra01_5b,ra10_5b),(g_A,ra01_A,ra10_A),(g_Pb,ra01_Pb_tz,ra10_Pb_tz),(g_Pb,ra01_Pb_zz,ra10_Pb_zz)] for tf in tfs_3pt_fit]
                        t=[t_2pta,t_2ptb]+t_3pt
                        return t
                    res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('fitPCAC',ens,str_diag,case,datmin))
                    fitres_PCAC[(ens,i_diags,case,datmin)]=res
                    if res is None:
                        continue
                    pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                    fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                fitres_PCAC[(ens,i_diags,case,'modelAvg')]=(pars_mean_MA,pars_err_MA,probs)
            
runFit_PCAC()

In [ ]:
def runFF(ptofas,ptofbs,FF,label=''):
    fig, axs = yu.getFigAxs(4,4,sharex='col')
    yu.addRowHeader(axs,['cA24: noJL','cA24: all','cA48: noJL','cA48: all'])
    yu.addColHeader(axs,['ratio','summation','2st-fit','joint'])
    
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    basekey=(pta,tuple(ofas),ptb,tuple(ofbs),FF,label)
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        xUnit=lat.a
        
        t0Lista=[t0 for t0,t in ens2choice[(ens,pta)]['choice']]
        tLista=[t for t0,t in ens2choice[(ens,pta)]['choice']]
        t0Listb=[t0 for t0,t in ens2choice[(ens,ptb)]['choice']]
        tListb=[t for t0,t in ens2choice[(ens,ptb)]['choice']]
        
        mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
        t_Fpi=92.9/lat.aInv
        t_aQ2=(qk**2-(EN-mN)**2)
        t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
        t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
        t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
        
        t_A2pNN_0=mN/t_Fpi
        t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
        t_52pNN_1=(lat.amu/lat.ZP[0])*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
        t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
        
        t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA[0]; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
        t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA[0]; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
        t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*(2*mN*(EN+mN)+2*qk**2)) * lat.ZA[0]; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
        t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*(2*mN*(EN+mN))) * lat.ZA[0]; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
        
        t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA[0]
        t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS[0]; t_G5b_01=t_G5_01 * t_52pNN_1
        t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA[0]; t_GP_01_tz_A=-(2*mN)/(EN-mN)
        t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA[0]; t_GP_01_zz_A=-(2*mN)/(EN-mN)

        t_PCAC_G5=(lat.amu/lat.ZP[0])/mN
        t_PCAC_GP=t_aQ2/(4*mN**2)
        
        pips={
            'GS+_00':[(lat.ZP[0],'id_j+','P0')],
            'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
            'J5_11':[(lat.ZS[0],'g5_j-','Pz')],
            'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
            'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
            'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
            
            'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
            'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
            'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
            'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
            
            # 'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
            # 'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA[0],'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA[0],'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP[0]*lat.ZS[0],'g5_j-','Pz')],
            
            # 'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            # 'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        
            'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
            
            # 'rPCAC2_1':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),(2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
        }[FF]
         
        ren={
            'GS+_00':lat.ZP,'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
            'G5b_01':lat.ZSbyZP,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,'rPCAC_1_Ptz':(1,1e-5),'rPCAC_1_Pzz':(1,1e-5)
        }[FF]
        def errRen(mean,err,ren):
            tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
            if FF in ['J5_11']:
                tcov+=1e-8
            tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
            return tmean,np.sqrt(np.diag(tcov))
                
        proj2mat={
            'P0':np.array([[1,0],[0,1]]),
            'Px':np.array([[0,1],[1,0]]),
            'Py':np.array([[0,-1j],[1j,0]]),
            'Pz':np.array([[1,0],[0,-1]]),
        }
        
        for i_diags,diags,str_diag in [(0,yu.diags_jLoopless,'loopless'),(1,yu.diags_all,'all')]:
            irow=i_ens*2+i_diags
            needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS+_00','J5_11']
            
            if ens in ['cA2.09.48'] and FF in FF2simone:
                t_mean,t_err=simone[FF2simone[FF]]
                axs[irow,icol].fill_between([(1-tfs[-1]//2-0.2)*xUnit,(tfs[-1]//2+0.2)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
                axs[irow,icol+1].fill_between([(tfs[0]-0.5)*xUnit,(tfs[-1]+0.5)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
                axs[irow,icol+2].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
                axs[irow,icol+3].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
               
            for case in ['noGEVP','GEVP']:
                pta,ofas=ptofas; ptb,ofbs=ptofbs
                if case=='noGEVP':
                    ofas=[ofas[0]]; ofbs=[ofbs[0]]
                
                def t_get(tf,t_insert):
                    t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                        for lb in ['l1','l2']] for la in ['l1','l2']]
                    return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                
                dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
                dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                        get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00'] else 'g5_j-']]
                datVEVpi0f=[data['VEV']['pi0f']]
                dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
                
                AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
                def func(dat):
                    t=yu.meanDeep(dat)
                    t3pt,t2pt,tVEVj,tVEVpi0f=t
                    
                    t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                    t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                    t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                    t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                    
                    for ind in range(len(t3pt)):
                        for ia,la in enumerate(['l1','l2']):
                            for ib,lb in enumerate(['l1','l2']): 
                                t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                    
                    eVals_al1,_=yu.GEVP(t2pt[0],1)
                    eVals_al2,_=yu.GEVP(t2pt[1],1)
                    eVals_a=(eVals_al1+eVals_al2)/2
                    Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                    
                    eVals_bl1,_=yu.GEVP(t2pt[2],1)
                    eVals_bl2,_=yu.GEVP(t2pt[3],1)
                    eVals_b=(eVals_bl1+eVals_bl2)/2
                    Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                    
                    _,eVecs_al1=yu.GEVP(t2pt[0],t0Lista,tList=tLista)
                    _,eVecs_al2=yu.GEVP(t2pt[1],t0Lista,tList=tLista)
                    _,eVecs_bl1=yu.GEVP(t2pt[2],t0Listb,tList=tListb)
                    _,eVecs_bl2=yu.GEVP(t2pt[3],t0Listb,tList=tListb)

                    eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                    eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni)
                    
                    t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                    t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                    
                    t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                    t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                    
                    if needsVEV:
                        proj=proj2mat[pips[0][2]]
                        t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                            np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                    
                    if FF not in ['rPCAC_1_Ptz','rPCAC_1_Pzz','rPCAC2_1']:
                        def t_get2(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                            return res
                        t=[(
                            t_get2(i)
                            -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                        )/np.sqrt(
                            t2pta_t[tf]*t2ptb_t[tf]*\
                            # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                            # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                            np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                        )
                        for i,tf in enumerate(tfs)]
                    else:
                        def t_get2_N(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                if i_pip<2:
                                    continue
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                            return res
                        def t_get2_D(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                if i_pip>=2:
                                    continue
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                            return res

                        t=[(
                            t_get2_N(i)/t_get2_D(i)
                        )
                        for i,tf in enumerate(tfs)]
                
                    t=yu.prefactorDeep(t,1)
                                
                    t_sum=np.array([np.sum(ele) for ele in t])
                    t_sumdif=(t_sum-np.roll(t_sum,1))/2
                    t+=[t_sumdif,t_sum]
                    
                    return t
                
                icol=0
                color={'noGEVP':'b','GEVP':'r'}[case]
                mfc={'noGEVP':'white','GEVP':None}[case]
                mean,err,_=yu.jackknife(dat,func,sl_key=(basekey,ens,str_diag,case,1))
                
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color=color,fmt=['s','d','o'][i_tf],mfc=mfc)  
                axs[irow,icol].set_ylim(ylimDic[FF])    
                                        
                i_shift=len(tfs)
                ind=i_shift
                tMean=mean[ind][1:]; tErr=err[ind][1:]
                tMean,tErr=errRen(tMean,tErr,ren)
                axs[irow,icol+1].errorbar(np.array(tfs[1:])*xUnit,tMean,tErr,color=color,fmt=['s','d','o'][i_tf],mfc=mfc)
                axs[irow,icol+1].set_ylim(ylimDic[FF])   

                if FF in ['J5_11']:
                    continue

                def func2(dat):
                    t=func(dat)[i_shift+1]
                    return t
                def fitfunc(g,a):
                    return g*np.array(tfs)+a
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,_=yu.fit(dat,func2,fitfunc,pars0=[1,0],sl_key=(basekey,ens,str_diag,case,2))
                t_mean=pars_mean[0]; t_err=pars_err[0]
                t_extend={'noGEVP':0,'GEVP':1}[case]
                t_mean,t_err=errRen(t_mean,t_err,ren)
                axs[irow,icol+1].fill_between([(tfs[0]-t_extend*0.25)*xUnit,(tfs[-1]+t_extend*0.25)*xUnit],t_mean-t_err,t_mean+t_err,color=color,alpha=0.2)
                # axs[irow,icol].fill_between([(1-tfs[-1]//2-t_extend*0.1)*xUnit,(tfs[-1]//2+t_extend*0.1)*xUnit],t_mean-t_err,t_mean+t_err,color=color,alpha=0.2)
                
                shift_chi2=(ylimDic[FF][1]-ylimDic[FF][0])/20
                FF2ind={'G5b_01':8,'GA_01':11,'GPb_01_tz':14,'GPb_01_zz':14}
                if FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
                    # joint fit
                    def estimator(x):
                        G5b=x[FF2ind['G5b_01']]
                        GA=x[FF2ind['GA_01']]
                        GPb=x[FF2ind['GPb_01_tz']]
                        ZSbyZPt=x[-2]
                        ZAt=x[-1]
                        rPCAC=(G5b/t_G5b_01*t_G5_01*t_PCAC_G5*ZSbyZPt + GPb/t_P2pNN_1*t_PCAC_GP*ZAt)/(GA*ZAt)
                        return np.array([rPCAC])
                    def errRen2(mean,cov):
                        N=len(mean)
                        tmean=np.concatenate([mean,[1,1]])
                        tcov=np.zeros((N+2,N+2))
                        tcov[:N,:N]=cov
                        tcov[N,N]=(lat.ZSbyZP[1]/lat.ZSbyZP[0])**2
                        tcov[N+1,N+1]=(lat.ZA[1]/lat.ZA[0])**2
                        return yu.propagateError(estimator,tmean,tcov)
                    fits=[]
                    for datmin in range(1,6):
                        res=fitres_PCAC[(ens,i_diags,case,datmin)]
                        if res is None:
                            continue
                        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                        tMean,tCov=errRen2(pars_mean,pars_cov)
                        tErr=np.sqrt(np.diag(tCov))
                        fits.append((tMean,tErr,chi2R_mean,Ndof))
                        axs[irow,icol+3].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
                        axs[irow,icol+3].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)
                        axs[irow,icol+3].set_ylim(ylimDic[FF])  
                    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                    tMean=pars_mean_MA[0]; tErr=pars_err_MA[0]
                    axs[irow,icol+3].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                    continue
                
                # 2st fit
                def func_base(dat):
                    t=yu.meanDeep(dat)
                    t3pt,t2pt,tVEVj,datVEVpi0f=t
                    
                    t2pt[0]=remove_pi0VEV(t2pt[0],datVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                    t2pt[1]=remove_pi0VEV(t2pt[1],datVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                    t2pt[2]=remove_pi0VEV(t2pt[2],datVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                    t2pt[3]=remove_pi0VEV(t2pt[3],datVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                    
                    for ind in range(len(t3pt)):
                        for ia,la in enumerate(['l1','l2']):
                            for ib,lb in enumerate(['l1','l2']): 
                                t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],datVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                    
                    _,eVecs_al1=yu.GEVP(t2pt[0],t0Lista,tList=tLista)
                    _,eVecs_al2=yu.GEVP(t2pt[1],t0Lista,tList=tLista)
                    _,eVecs_bl1=yu.GEVP(t2pt[2],t0Listb,tList=tListb)
                    _,eVecs_bl2=yu.GEVP(t2pt[3],t0Listb,tList=tListb)

                    eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                    eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni)
                    
                    t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                    t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                    
                    t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                    t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                    
                    if needsVEV:
                        proj=proj2mat[pips[0][2]]
                        t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                            np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                            
                    def t_get2(i):
                        res=0
                        for i_pip,pip in enumerate(pips):
                            prefactor,insert,proj=pip
                            proj=proj2mat[proj]
                            proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                            res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                        return res
                    t=[t2pta_t,t2ptb_t]+[ t_get2(i)-(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0) for i,tf in enumerate(tfs)]
                
                    t=yu.prefactorDeep(t,1)
                    return t
                
                if pta==ptb:
                    pars0=np.concatenate([ens2p0[(ens,pta,case)],[np.mean(ylimDic[FF]),1]])
                    fits=[]
                    for datmin in range(1,6):
                        datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                        tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                        tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                        tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                        tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf//2+1) for tf in tfs_3pt_fit}
                        Ny_2pt=len(tfs_2pta_fit)+len(tfs_2ptb_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                        Ny=Ny_2pt+Ny_3pt
                        def func(dat):
                            t=func_base(dat)
                            t[0]=t[0][tfs_2pta_fit]
                            for i,tf in enumerate(tfs):
                                t[i+2]=t[i+2][tcs_3pt_fit[tf]-1]
                            t=[t[0]]+t[2:]
                            return t
                        def fitfunc(E0a,c0a,dE1a,rc1a,g,ra01):
                            t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                            t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0a,g*np.sqrt(c0a*c0a),dE1a,dE1a,ra01,ra01,0) for tf in tfs_3pt_fit]
                            t=[t_2pta]+t_3pt
                            return t
                        res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=(basekey,ens,str_diag,case,2,datmin))
                        if res is None:
                            continue
                        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                        
                        tMean=[pars_mean[4]]; tErr=[pars_err[4]]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol+2].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
                        axs[irow,icol+2].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)
                        axs[irow,icol+2].set_ylim(ylimDic[FF])  
                        
                    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                    tMean=pars_mean_MA[4]; tErr=pars_err_MA[4]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol+2].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                else:                                      
                    pars0=np.concatenate([ens2p0[(ens,pta,case)],ens2p0[(ens,ptb,case)],[np.mean(ylimDic[FF]),1,1]])
                    fits=[]
                    for datmin in range(1,6):
                        datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                        tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                        tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                        tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                        tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                        Ny_2pt=len(tfs_2pta_fit)+len(tfs_2ptb_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                        Ny=Ny_2pt+Ny_3pt
                        def func(dat):
                            t=func_base(dat)
                            t[0]=t[0][tfs_2pta_fit]
                            t[1]=t[1][tfs_2ptb_fit]
                            for i,tf in enumerate(tfs):
                                t[i+2]=t[i+2][tcs_3pt_fit[tf]-1]
                            return t
                        def fitfunc(E0a,c0a,dE1a,rc1a,E0b,c0b,dE1b,rc1b,g,ra01,ra10):
                            t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                            t_2ptb=func_C2pt_2st(tfs_2ptb_fit,E0b,c0b,dE1b,rc1b)
                            t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0b,g*np.sqrt(c0a*c0b),dE1a,dE1b,ra01,ra10,0) for tf in tfs_3pt_fit]
                            t=[t_2pta,t_2ptb]+t_3pt
                            return t
                        res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=(basekey,ens,str_diag,case,3,datmin))
                        if res is None:
                            continue
                        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                        
                        tMean=[pars_mean[8]]; tErr=[pars_err[8]]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol+2].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
                        axs[irow,icol+2].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)
                        axs[irow,icol+2].set_ylim(ylimDic[FF])  
                        
                    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                    tMean=pars_mean_MA[8]; tErr=pars_err_MA[8]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol+2].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                
                # joint
                if FF in ['G5b_01','GA_01','GPb_01_tz','GPb_01_zz']:
                    ind=FF2ind[FF]
                    fits=[]
                    for datmin in range(1,6):
                        res=fitres_PCAC[(ens,i_diags,case,datmin)]
                        if res is None:
                            continue
                        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                        tMean=[pars_mean[ind]]; tErr=[pars_err[ind]]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol+3].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
                        axs[irow,icol+3].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)
                        axs[irow,icol+3].set_ylim(ylimDic[FF])  
                    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                    tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol+3].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                          
    plt.tight_layout()
    plt.savefig('fig/'+FF+label+'.pdf')
    plt.close()
    print(FF)
 
for FF in ['GS+_00','GA_00_zz']:
    runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),FF)
for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
    runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF)
    
for FF in ['G5b_01','GA_01','GPb_01_tz','GPb_01_zz']:
    runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF)
for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
    runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF)